In [0]:
%pip install yahooquery

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import pickle
import requests
import numpy as np
from yahooquery import Ticker
import datetime
from datetime import date
import urllib.request
import sys
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import re

In [0]:
%run "/Shared/WeatherWatts/config/kafka keys"

In [0]:
%run "/Shared/WeatherWatts/config/api keys"

In [0]:
delta1 = datetime.timedelta(days=2)
delta2  =datetime.timedelta(days=4)
delta3 = datetime.timedelta(days=-3)
delta4 = datetime.timedelta(days=1)
delta5 = datetime.timedelta(days=3)
today = date.today()
one_day_ahead = today + delta4
two_day_ahead = today + delta1
four_days_ahead = today + delta2
three_days_behind = today +delta3
three_days_ahead = today + delta5

In [0]:
holidays = spark.read.csv('/mnt/weatherwatts/holiday.csv', header=True)

In [0]:
holidays = holidays.toPandas()

In [0]:
ticker = Ticker('XLF')

In [0]:
XLE = ticker.history(period='3mo', interval='1d', adj_timezone=False)['adjclose'].unstack().T

In [0]:
future = pd.date_range(str(one_day_ahead), str(four_days_ahead), freq='1d', tz='UTC')

In [0]:
future_df = pd.DataFrame(index=future)

In [0]:
XLE = pd.concat([XLE, future_df])

In [0]:
from datetime import date
XLE.index = pd.to_datetime(XLE.index, utc=True)
XLE['year'] = XLE.index.year
XLE['dayofyear'] =XLE.index.dayofyear

In [0]:
XLE.rename(columns={'XLF':'Adj Close'}, inplace=True)

In [0]:
XLE['Adj_Close_3D_lag'] = XLE['Adj Close'].shift(freq='3D')
XLE['Adj_Close_30D_lag'] = XLE['Adj Close'].shift(freq='28D')
XLE['Adj_Close_60D_lag'] = XLE['Adj Close'].shift(freq='56D')
XLE['Adj_Close_30D_lag'] = XLE['Adj_Close_30D_lag'].ffill()
XLE['Adj_Close_60D_lag'] = XLE['Adj_Close_60D_lag'].ffill()
XLE['Adj_Close_3D_lag'] = XLE['Adj_Close_3D_lag'].ffill()


In [0]:
locations2 = ['Los%20Angeles', 'Tampa', 'Austin','new%20york%20city']

In [0]:
weather_df = pd.DataFrame()
try:
    for loc in locations2:
        ResultBytes = urllib.request.urlopen(f"https://weather.visualcrossing.com/VisualCrossingWebServices/"
                                             f"rest/services/timeline/{loc}/{three_days_behind}/{four_days_ahead}?"
                                             f"unitGroup=us&include=fcst%2Chours%2Cobs&"
                                             f"key={w_key}&contentType=json")

      # Parse the results as JSON
        jsonData = json.load(ResultBytes)
        for i in range(len(jsonData['days'])):
            day = pd.DataFrame(jsonData['days'][i]['hours'])
            day['name'] = loc.replace('%20',' ')
            weather_df = pd.concat([weather_df,day])

except urllib.error.HTTPError  as e:
  ErrorInfo= e.read().decode()
  print('Error code: ', e.code, ErrorInfo)
  sys.exit()
except  urllib.error.URLError as e:
  ErrorInfo= e.read().decode()
  print('Error code: ', e.code,ErrorInfo)
  sys.exit()

In [0]:
locations =[ "CISO", "ERCO","NYIS","FLA"]

In [0]:
def json_to_dateframe(response):
    return pd.DataFrame(response.json()['response']['data'][1:], columns= response.json()['response']['data'][0])

In [0]:
from time import sleep
energy = pd.DataFrame()
for loc in locations:
    try:
        url = f'https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&facets[respondent][]={loc}&facets[type][]=D&facets[type][]=NG&start={str(three_days_behind)}T00&end={str(today)}T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={key}'
        response = requests.get(url)
        df = json_to_dateframe(response)
        energy = pd.concat([energy, df])
    except:
        print('error')

In [0]:
demand = energy.query("type == 'D' ")

In [0]:
demand['respondent-name'] = demand['respondent-name'].replace('Florida', 'Florida Power and Light').copy()
demand['period'] = pd.to_datetime(demand.period)
demand['period'] = pd.to_datetime(demand.period)

In [0]:
respondents =['Electric Reliability Council of Texas, Inc.' ,
              'New York Independent System Operator',
              'Florida Power and Light' ,
               'California Independent System Operator',]

In [0]:
demand['name'] = demand['respondent-name']

In [0]:
demand['name'] = demand['name'].replace(respondents[3], 'Los Angeles')
demand['name'] = demand['name'].replace(respondents[2], 'Tampa')
demand['name'] = demand['name'].replace(respondents[0], 'Austin')
demand['name'] = demand['name'].replace(respondents[1], 'new york city')

In [0]:
weather_df['period'] = pd.to_datetime(weather_df.datetimeEpoch, unit='s')

In [0]:
demand_weather = pd.merge(demand,weather_df, how='right', on=['name','period'],suffixes=('','2'))
demand_weather = demand_weather.set_index(demand_weather.period)
demand_weather= demand_weather.drop(columns=['type','type-name','datetime','datetimeEpoch','stations','solarenergy','source','severerisk','preciptype','windgust','value-units','respondent','icon','conditions'])

In [0]:
demand_weather['year'] = demand_weather.index.year
demand_weather['month'] = demand_weather.index.month
demand_weather['days'] = demand_weather.index.dayofweek
demand_weather['hour'] = demand_weather.index.hour
demand_weather['quarter'] = demand_weather.index.quarter
demand_weather['dayofyear'] = demand_weather.index.dayofyear
demand_weather["Is Weekend"] = demand_weather.index.day_name().isin(['Saturday', 'Sunday'])

In [0]:
demand_weather = pd.merge(demand_weather,XLE.iloc[:,1:],how='left', on=['year','dayofyear'])

In [0]:
holidays['year'] = holidays['year'].astype('int')
holidays['dayofyear'] = holidays['dayofyear'].astype('int')

In [0]:
holidays =holidays.drop('Date',axis=1)
demand_weather = pd.merge(demand_weather,holidays, how='outer', on=['year','dayofyear'])

In [0]:
demand_weather['solarradiation'] = demand_weather['solarradiation'].bfill()
demand_weather['uvindex'] = demand_weather['uvindex'].bfill()
demand_weather['snow'] = demand_weather['snow'].bfill()

In [0]:
demand_weather.loc[demand_weather['name'] == 'Los Angeles', 'respondent-name'] = 'California Independent System Operator'
demand_weather.loc[demand_weather['name'] == 'new york city', 'respondent-name'] = 'New York Independent System Operator'
demand_weather.loc[demand_weather['name'] == 'Tampa', 'respondent-name'] = 'Florida Power and Light'
demand_weather.loc[demand_weather['name'] == 'Austin', 'respondent-name'] = 'Electric Reliability Council of Texas, Inc.'

In [0]:
demand_weather = demand_weather.set_index('period')
demand_weather.index = pd.to_datetime(demand_weather.index)
demand_weather.rename(columns={'respondent-name':'respondent','pressure': 'sealevelevelpressure'}, inplace=True)

In [0]:
demand_weather = pd.get_dummies(demand_weather, columns=['Holiday','Is Weekend'], drop_first=True, dtype=float)
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in demand_weather.columns}
new_n_list = list(new_names.values())
# [LightGBM] Feature appears more than one time.
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
demand_weather = demand_weather.rename(columns=new_names)

In [0]:
NY = demand_weather.query("respondent== 'New York Independent System Operator'")
FL = demand_weather.query("respondent == 'Florida Power and Light'")
CA = demand_weather.query("respondent ==  'California Independent System Operator'")
TX = demand_weather.query("respondent ==  'Electric Reliability Council of Texas, Inc.'")

In [0]:
TX = TX[~TX.index.duplicated(keep='first')]
NY = NY[~NY.index.duplicated(keep='first')]
CA = CA[~CA.index.duplicated(keep='first')]
FL = FL[~FL.index.duplicated(keep='first')]

In [0]:
NY['demand_3d_lag'] = NY['value'].shift(96, freq='h')
FL['demand_3d_lag'] = FL['value'].shift(96, freq='h')
CA['demand_3d_lag'] = CA['value'].shift(96, freq='h')
TX['demand_3d_lag'] = TX['value'].shift(96, freq='h')

In [0]:
NY['Temp_24_hours_ahead'] = NY['temp'].shift(-24, freq='h')
FL['Temp_24_hours_ahead'] = FL['temp'].shift(-24 ,freq='h')
CA['Temp_24_hours_ahead'] = CA['temp'].shift(-24, freq='h')
TX['Temp_24_hours_ahead'] = TX['temp'].shift(-24, freq='h')

In [0]:
# TX  = TX.iloc[(TX.index >= str(today)) & (TX.index <= str(four_days_ahead))]
# NY  = NY.iloc[(NY.index >= str(today)) & (NY.index <= str(four_days_ahead))]
# CA  = CA.iloc[(CA.index >= str(today)) & (CA.index <= str(four_days_ahead))]
# FL  = FL.iloc[(FL.index >= str(today)) & (FL.index <= str(four_days_ahead))]

In [0]:
TX= TX.iloc[(TX.index > str(three_days_ahead))][:24]
NY= NY.iloc[(NY.index > str(three_days_ahead))][:24]
CA= CA.iloc[(CA.index > str(three_days_ahead))][:24]
FL= FL.iloc[(FL.index > str(three_days_ahead))][:24]

In [0]:
Features = TX.columns[3:]

In [0]:
TX_X = TX[Features]
NY_X = NY[Features]
CA_X = CA[Features]
FL_X = FL[Features]

In [0]:
TX_X = TX_X.ffill()
NY_X = NY_X.ffill()
CA_X = CA_X.ffill()
FL_X = FL_X.ffill()

In [0]:
TX_X = TX_X.dropna(subset=['demand_3d_lag'])
NY_X = NY_X.dropna(subset=['demand_3d_lag'])
CA_X = CA_X.dropna(subset=['demand_3d_lag'])
FL_X = FL_X.dropna(subset=['demand_3d_lag'])

In [0]:
TX_X['State'] = 'TX'
NY_X['State'] = 'NY'
CA_X['State'] = 'CA'
FL_X['State'] = 'FL'

In [0]:
final_df = pd.concat([TX_X,NY_X,FL_X,CA_X])

In [0]:
final_df['Date'] = final_df.index

In [0]:
final_df = spark.createDataFrame(final_df)

In [0]:
from pyspark.sql.functions import *
import json
json_final = final_df.toJSON().map(lambda str_json: json.loads(str_json)).collect()

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
import uuid
import confluent_kafka as kafka
import confluent_kafka.admin as kadmin

KWARGS = {
    'bootstrap.servers' : confluentBootstrapServers,
    'sasl.username'     : confluentApiKey,
    'sasl.password'     : confluentSecret,
    'sasl.mechanism'    : 'PLAIN',
    'security.protocol' : 'SASL_SSL',
    'auto.offset.reset' : 'earliest',
    'session.timeout.ms': 45000,
    'group.id'          : str(uuid.uuid4())
}

# Create admin client, for adding topics
admin_client = kadmin.AdminClient(KWARGS)

# Create producer, for sending messages
producer = kafka.Producer(KWARGS)

In [0]:
TOPICS = ["future_data"]
try:
    admin_client.delete_topics(topics=TOPICS)
    print("Topic Deleted Successfully")
    sleep(30)
except:
    print("Topic Doesn't Exist")

Topic Deleted Successfully


In [0]:
TOPICS = ["future_data"]

kafka_topics = [kadmin.NewTopic(topic, 1) for topic in TOPICS] 
admin_client.create_topics(kafka_topics)

created_topics = admin_client.list_topics().topics
for t in TOPICS:
    sleep(30)
    print(f'{t}: {t in created_topics}')

future_data: False


In [0]:
len(json_final)

Out[56]: 272

In [0]:
for i in range(len(json_final)): 
    # print(json_final[i])
    producer.produce(TOPICS[0],json.dumps(json_final[i]))
    producer.flush()

In [0]:
from datetime import datetime, timedelta

# Create Kafka Consumer ====================================================
consumer = kafka.Consumer(KWARGS)
consumer.subscribe(TOPICS) 


# Consume messages =========================================================
max_messages, timeout_in_sec = 10000, 10 # Optional exit conditions

start_time = datetime.now()
mesg = []

while True:
    try:
        msg = consumer.poll(8.0) # .poll(poll_timeout_in_seconds)
        if msg is None:
            break
            raise Exception('Message value is None')
            
        elif msg.error() is not None:
            raise Exception(str(msg.error()))
        else:
            msg_val = json.loads('{}'.format(msg.value().decode('utf-8')))
            mesg.append(msg_val)
    except Exception as e:
        cons_results['failures'].append((str(msg), e))

consumer.close()

In [0]:
len(mesg)

Out[59]: 272

In [0]:
df_c= pd.DataFrame(mesg)

In [0]:
df_ny = df_c.query('State == "NY"')
df_tx = df_c.query('State == "TX"')
df_ca = df_c.query('State == "CA"')
df_fl = df_c.query('State == "FL"')

In [0]:
df_ny = spark.createDataFrame(df_ny)
df_tx = spark.createDataFrame(df_tx)
df_ca = spark.createDataFrame(df_ca)
df_fl = spark.createDataFrame(df_fl)

In [0]:
%run "/Shared/WeatherWatts/config/SQL Keys"

In [0]:
df_ny.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableNY) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [0]:
df_ca.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableCA) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [0]:
df_tx.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableTX) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [0]:
df_fl.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", tableFL) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()